In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Initialize the app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "People Analytics Dashboard"
app.config.suppress_callback_exceptions = True

# Load and preprocess data
df = pd.read_csv("data/data.csv")
df["Date of Hire"] = pd.to_datetime(df["Date of Hire"], errors="coerce")
df["Date of Termination"] = pd.to_datetime(df["Date of Termination"], errors="coerce")

# Generate data for headcount and attrition
hire = df["Date of Hire"].value_counts().sort_index()
term = df["Date of Termination"].value_counts().sort_index()
emp_count = hire.sub(term, fill_value=0).cumsum()
attrition_rate = (term / emp_count).fillna(0) * 100

# Recruitment Data
recruitment_data = pd.DataFrame({
    "Region": ["North America", "Europe", "Asia", "South America", "Africa"],
    "Job Board": ["LinkedIn", "Indeed", "Glassdoor", "Internal Referrals", "Career Page"],
    "Job Count": [150, 120, 300, 80, 50],
    "Latitude": [37.7749, 48.8566, 28.6139, -23.5505, -1.2921],
    "Longitude": [-122.4194, 2.3522, 77.2090, -46.6333, 36.8219]
})

# Mock data for HR Helpdesk
ticket_data = pd.DataFrame({
    "Category": ["Payroll Issues", "Leave Requests", "IT Access", "Policy Queries", "Others"],
    "Count": [40, 20, 15, 15, 10]
})

# Mock prediction data
np.random.seed(42)
actual = np.random.randint(20, 50, size=12)
predicted_algo1 = actual + np.random.randint(-5, 5, size=12)
predicted_algo2 = actual + np.random.randint(-10, 10, size=12)

# Prediction accuracy calculations
mae_algo1 = np.mean(np.abs(actual - predicted_algo1))
mae_algo2 = np.mean(np.abs(actual - predicted_algo2))
accuracy_algo1 = 100 - (mae_algo1 / np.mean(actual)) * 100
accuracy_algo2 = 100 - (mae_algo2 / np.mean(actual)) * 100

# Layout styling
layout_style = {
    "plot_bgcolor": "#000",
    "paper_bgcolor": "#000",
    "font": {"color": "white"},
    "xaxis": {"gridcolor": "gray", "title": "Date"},
    "yaxis": {"gridcolor": "gray"}
}
tabs_style = {"fontSize": 16, "backgroundColor": "#000", "color": "white", "padding": "10px"}
selected_tab_style = {"fontSize": 16, "backgroundColor": "#fff", "color": "#000", "padding": "10px"}

# Header
header = html.Div([
    html.H2(
        "People Analytics Dashboard",
        style={"color": "#FFE600", "textAlign": "left", "fontWeight": "bold", "fontSize": "28px", "flexGrow": "1"}
    ),
    html.Img(
        src="/assets/sample logo-1.jpg",  # Ensure logo is in the 'assets' folder
        style={"height": "50px", "float": "right", "marginRight": "10px"}
    )
], style={"backgroundColor": "#000", "padding": "10px", "display": "flex", "alignItems": "center"})

# Tabs content
tabs_content = {
    "headcount": html.Div([
        html.Label("Filter by Department:", style={"color": "white"}),
        dcc.Dropdown(
            id="department_filter",
            options=[{"label": dept, "value": dept} for dept in df["Department"].dropna().unique()],
            placeholder="Select Department",
            style={"width": "50%", "color": "black"}
        ),
        dcc.Graph(id="employee_count_chart"),
        dcc.Graph(
            id="attrition_chart",
            figure={
                "data": [
                    go.Scatter(
                        x=emp_count.index,
                        y=attrition_rate,
                        mode="lines+markers",
                        name="Attrition Rate (%)",
                        line=dict(color="#FFE600")
                    )
                ],
                "layout": go.Layout(title="Monthly Attrition Rate", **layout_style)
            }
        ),
        dcc.Graph(
            id="additional_chart",
            figure={
                "data": [
                    go.Bar(
                        x=hire.index,
                        y=hire.values,
                        name="New Hires",
                        marker=dict(color="#FFE600")
                    ),
                    go.Bar(
                        x=term.index,
                        y=term.values,
                        name="Terminations",
                        marker=dict(color="#FF6347")
                    )
                ],
                "layout": go.Layout(title="Hires vs Terminations", **layout_style)
            }
        )
    ]),
    "recruitment": html.Div([
        html.Label("Filter by Region:", style={"color": "white"}),
        dcc.Dropdown(
            id="region_filter",
            options=[{"label": region, "value": region} for region in recruitment_data["Region"]],
            placeholder="Select Region",
            style={"width": "50%", "color": "black"}
        ),
        dcc.Graph(id="recruitment_map"),
        dcc.Graph(
            id="job_board_chart",
            figure={
                "data": [
                    go.Pie(
                        labels=recruitment_data["Job Board"],
                        values=recruitment_data["Job Count"],
                        marker=dict(colors=["#FFE600", "#0047AB", "#808080", "#333333", "#AAAAAA"])
                    )
                ],
                "layout": go.Layout(title="Job Board Distribution", **layout_style)
            }
        )
    ]),
    "helpdesk": html.Div([
        dcc.Graph(
            id="helpdesk_chart",
            figure={
                "data": [
                    go.Pie(
                        labels=ticket_data["Category"],
                        values=ticket_data["Count"],
                        marker=dict(colors=["#FFE600", "#0047AB", "#808080", "#333333", "#AAAAAA"])
                    )
                ],
                "layout": go.Layout(title="HR Helpdesk Ticket Distribution", **layout_style)
            }
        )
    ]),
    "predictions": html.Div([
        html.Div([
            html.H4(f"Algorithm 1 Accuracy: {accuracy_algo1:.2f}%", style={"color": "#FFE600"}),
            html.H4(f"Algorithm 2 Accuracy: {accuracy_algo2:.2f}%", style={"color": "#0047AB"})
        ], style={"display": "flex", "justifyContent": "space-around", "marginBottom": "20px"}),
        dcc.Graph(
            id="prediction_chart",
            figure={
                "data": [
                    go.Scatter(
                        x=np.arange(12),
                        y=actual,
                        mode="lines+markers",
                        name="Actual",
                        line=dict(color="#FFE600")
                    ),
                    go.Scatter(
                        x=np.arange(12),
                        y=predicted_algo1,
                        mode="lines+markers",
                        name="Predicted Algo 1",
                        line=dict(color="#0047AB")
                    ),
                    go.Scatter(
                        x=np.arange(12),
                        y=predicted_algo2,
                        mode="lines+markers",
                        name="Predicted Algo 2",
                        line=dict(color="#FF6347")
                    )
                ],
                "layout": go.Layout(title="Prediction vs Actual", **layout_style)
            }
        )
    ]),
    "performance": html.Div([
        dcc.Graph(
            id="performance_chart",
            figure={
                "data": [
                    go.Bar(
                        x=["Q1", "Q2", "Q3", "Q4"],
                        y=[85, 90, 88, 92],
                        name="Performance Score",
                        marker=dict(color="#FFE600")
                    )
                ],
                "layout": go.Layout(title="Quarterly Performance", **layout_style)
            }
        )
    ])
}

# App layout
app.layout = html.Div([
    header,
    dcc.Tabs(
        id="tabs",
        value="headcount",
        children=[
            dcc.Tab(label="Headcount & Attrition", value="headcount", style=tabs_style, selected_style=selected_tab_style),
            dcc.Tab(label="Recruitment", value="recruitment", style=tabs_style, selected_style=selected_tab_style),
            dcc.Tab(label="HR Helpdesk", value="helpdesk", style=tabs_style, selected_style=selected_tab_style),
            dcc.Tab(label="Predictions", value="predictions", style=tabs_style, selected_style=selected_tab_style),
            dcc.Tab(label="Performance", value="performance", style=tabs_style, selected_style=selected_tab_style)
        ]
    ),
    html.Div(id="tab_content", style={"backgroundColor": "#000", "padding": "20px"})
])

# Callbacks
@app.callback(Output("tab_content", "children"), [Input("tabs", "value")])
def render_tab(tab):
    return tabs_content.get(tab, html.Div("Content not available.", style={"color": "white"}))

@app.callback(
    Output("employee_count_chart", "figure"),
    [Input("department_filter", "value")]
)
def update_employee_count_chart(department):
    filtered_df = df if not department else df[df["Department"] == department]
    filtered_hire = filtered_df["Date of Hire"].value_counts().sort_index()
    filtered_term = filtered_df["Date of Termination"].value_counts().sort_index()
    filtered_emp_count = filtered_hire.sub(filtered_term, fill_value=0).cumsum()

    return {
        "data": [
            go.Scatter(
                x=filtered_emp_count.index,
                y=filtered_emp_count.values,
                mode="lines+markers",
                name="Employee Count",
                line=dict(color="#FFE600")
            )
        ],
        "layout": go.Layout(title="Employee Count Over Time", **layout_style)
    }

@app.callback(
    Output("recruitment_map", "figure"),
    [Input("region_filter", "value")]
)
def update_recruitment_map(region):
    filtered_data = recruitment_data if not region else recruitment_data[recruitment_data["Region"] == region]

    return {
        "data": [
            go.Scattergeo(
                lon=filtered_data["Longitude"],
                lat=filtered_data["Latitude"],
                text=filtered_data["Job Board"] + ": " + filtered_data["Job Count"].astype(str),
                mode="markers",
                marker=dict(size=filtered_data["Job Count"] / 10, color="#FFE600")
            )
        ],
        "layout": go.Layout(
            title="Recruitment Statistics by Region",
            geo=dict(showframe=False, showcoastlines=True, projection_type="equirectangular", bgcolor="#000"),
            paper_bgcolor="#000",
            font={"color": "white"}
        )
    }

if __name__ == "__main__":
    app.run_server(debug=True, port=1625)